In [1]:
import pyspark
import json
from pyspark.sql.functions import udf, from_json
from pyspark.sql.types import StructType, StructField, StringType

In [2]:
spark

In [3]:
def purchase_sword_event_schema():
    """
    root
    |-- Accept: string (nullable = true)
    |-- Host: string (nullable = true)
    |-- User-Agent: string (nullable = true)
    |-- event_type: string (nullable = true)
    |--
    |-- timestamp: string (nullable = true)
    """
    return StructType([
        StructField("Accept", StringType(), True),
        StructField("Host", StringType(), True),
        StructField("User-Agent", StringType(), True),
        StructField("event_type", StringType(), True),
    ])

@udf('boolean')
def is_sword_purchase(event_as_json):
    """udf for filtering events
    """
    event = json.loads(event_as_json)
    if event['event_type'] == 'purchase_sword':
        return True
    return False

@udf('string')
def event_type(event_as_json):
    event = json.loads(event_as_json)
    return event['event_type']

In [4]:
raw_events = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "events") \
    .load()
    
event_types = raw_events.select(event_type(raw_events.value.cast('string')).alias('event_type')) \
    .groupby('event_type') \
    .count()
    
sword_purchases = raw_events \
    .filter(is_sword_purchase(raw_events.value.cast('string'))) \
    .select(raw_events.value.cast('string').alias('raw_event'),
            raw_events.timestamp.cast('string'),
            from_json(raw_events.value.cast('string'),
                      purchase_sword_event_schema()).alias('json')) \
    .select('raw_event', 'timestamp', 'json.*')
    
query = event_types \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()
    
sink = sword_purchases \
    .writeStream \
    .format("parquet") \
    .option("checkpointLocation", "/tmp/checkpoints_for_sword_purchases") \
    .option("path", "/tmp/sword_purchases") \
    .trigger(processingTime="20 seconds") \
    .start()
    
    
query.awaitTermination()
sink.awaitTermination()

StreamingQueryException: 'Job aborted due to stage failure: Task 0 in stage 5.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5.0 (TID 206, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):\n  File "/spark-2.2.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main\n    process()\n  File "/spark-2.2.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process\n    serializer.dump_stream(func(split_index, iterator), outfile)\n  File "/spark-2.2.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/serializers.py", line 220, in dump_stream\n    self.serializer.dump_stream(self._batched(iterator), stream)\n  File "/spark-2.2.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/serializers.py", line 138, in dump_stream\n    for obj in iterator:\n  File "/spark-2.2.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/serializers.py", line 209, in _batched\n    for item in iterator:\n  File "<string

In [5]:
import redis
r = redis.Redis(host='redis', port='6379')

In [9]:
r.set('user1', json.dumps({"Guild": None, "Inventory": {}}).encode())
r.get('user1')

b'{"Guild": null, "Inventory": {}}'

In [10]:
json.loads(r.get('user1'))

{'Guild': None, 'Inventory': {}}